In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AlignmentCalculator
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7520243908438134038"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath("./tmp/video/7520243908438134038.mp4")
audio_path = os.path.abspath("./tmp/video/7520243908438134038.wav")

transcript = """
Știai că România vrea să vândă portul Constanța pentru a reconstrui Ucraina?
 Sună nebunesc? Din păcate e real.
 În cercurile guvernamentale de la București se discută tot mai intens despre vânzarea portului Constanța,
 iar banii obținuți ar urma să fie direcționați către reconstrucția Ucrainei.
 Nu este o glumă, vorbim despre cel mai important port românesc,
 o poartă strategică la Marea Neagră, cu valoare geo-economică și militară uriașă.
 Și se ia în calcul cedarea acestuia, în plină criză globală,
 în favoarea unui plan de solidaritate internațional.
 Întrebarea dură este, solidaritate cu cine și pe spatele cui?
 Ce câștigă România concret din această tranzacție?
 Cine va controla portul după vânzare?
 Cum ajută acest lucru românii care plătesc taxe, facturi și rate tot mai mari?
 În timp ce alte state își apără infrastructura vitală,
 România pare gata să renunțe la ea în numele unor interese geopolitice impuse din afară.
 Nu e doar o problemă economică, e o problemă de suveranitate națională.
 Tu ce crezi? România trebuie să ajute cu orice preț chiar dacă își vinde viitorul?
 Sau ar trebui să punem pe primul loc?
 interesul național, distribuie ca să afle toți ce se pregătește în tăcere.
"""

sentiment_results = {
    'positive': 0.15972021222114563,
    'neutral': 0.0,
    'negative': 0.840279757976532
 }
description = 'Portul Constanța, una dintre cele mai importante infrastructuri strategice ale României, ar putea fi vândut. Guvernul discută ca banii să fie direcționați spre reconstrucția Ucrainei. E o decizie corectă? Sau o  a interesului național #Romania2025 #PortulConstanta #Ucraina #Suveranitate #DistribuieAdevărul'

In [6]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7520243908438134038.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video alleges that Romania is considering selling its strategic Port of Constanța to fund Ukraine's reconstruction. It questions the wisdom of this decision, framing it as potentially detrimental to Romania's national interests and sovereignty, especially during a global crisis. The speaker implies that this move might be driven by external geopolitical interests rather than Romania's own benefit, asking whether Romania should help at any cos

{'summary': "The video alleges that Romania is considering selling its strategic Port of Constanța to fund Ukraine's reconstruction. It questions the wisdom of this decision, framing it as potentially detrimental to Romania's national interests and sovereignty, especially during a global crisis. The speaker implies that this move might be driven by external geopolitical interests rather than Romania's own benefit, asking whether Romania should help at any cost, even at the expense of its future, or prioritize its national interests.",
 'identified_subjects': [{'subject': 'romania', 'stance': -0.8},
  {'subject': 'ukraine', 'stance': 0.5},
  {'subject': 'european union', 'stance': -0.7},
  {'subject': 'national sovereignty', 'stance': -0.9}],
 'overall_alignment': -0.95}

In [7]:
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: -0.95 
final_alignment: -0.7701530612244897 
deterministic_alignment: -0.23061224489795915 
alignment_conflict: 0.7193877551020408
